## Struktury danych

In [12]:
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y

In [13]:
class Segment:
    def __init__(self,p,q):
        
        if p.x < q.x:
            left, right=p, q
        else:
            left, right=q, p
                    
        self.left=left
        self.right=right
        self.a=(right.y-left.y)/(right.x-left.x) # zakladam ze nie ma pionowych odcinkow
        self.b=left.y-left.x*self.a
        
    def get_y(self,x):
        return self.a*x+self.b


In [14]:
class Trapezoid:
    def __init__(self, top, bottom, leftp, rightp):

        self.top = top
        self.bottom = bottom
        self.leftp = leftp
        self.rightp = rightp
        
        self.dnode = None # wskaznik do węzła w grafie wyszukiwań

        # wskazniki do sasiadow
        self.upper_left = None
        self.lower_left = None
        self.upper_right = None
        self.lower_right = None


In [15]:
class DNode:
    def __init__(self, node_type, label, left=None, right=None, above = None, below = None):
        """
        node_type: Typ węzła ('x', 'y' lub 'leaf')
        label: Punkt, odcinek lub trapez, zależnie od typu węzła
        left: Lewy potomek
        right: Prawy potomek
        """
        self.node_type = node_type  # Typ węzła: 'x', 'y', 'leaf'
        self.label = label          # Punkt, odcinek lub trapez
        self.left = left            # Lewy potomek
        self.right = right          # Prawy potomek
        self.above = above
        self.below = below
        self.parents=[]  # lista rodziców potrzebna do zastępowania trapezów w grafie wyszukiwań na inne węzły
        # potrzebne tylko w przypadku trapezów, ale nie chce mi się tego wydzielać do osobnej klasy
        
        def replace(self, new_node):
            for parent in self.parents:
                if parent.left == self:
                    parent.left = new_node
                elif parent.above == self:
                    parent.above = new_node
                elif parent.below == self:
                    parent.below = new_node
                else:
                    parent.right = new_node
                new_node.parents.append(parent)

    
class DTree:
    def __init__(self):
        self.root = None

    def query(self, point,a=None):
        """
        Znajduje trapez, który zawiera dany punkt.
        point: (x, y) - współrzędne punktu
        Zwraca: Trapez (etykieta liścia), który zawiera punkt
        """
        current = self.root
        while current:
            if current.node_type == 'x':
                # Testujemy po współrzędnej x
                if point.x < current.label.x:
                    current = current.left                    
                else:
                    current = current.right

            elif current.node_type == 'y':
                # Testujemy położenie względem odcinka
                segment = current.label
                y_on_line = segment.get_y(point.x)
                if point.y > y_on_line:
                    current = current.above
                    
                #NIE WIEM CZY TO JEST WGL POTRZEBNE?
                # elif point.y==y_on_line and a is not None:   # potrzebne do wyznaczania strefy dla s_i, jeżeli początek dodawanego odcinka leży na sprawdzanej linii
                #     # nie wiem czy to rozwiazuje problem  
                #     if segment.a>a:
                #         current = current.below
                #     else:
                #         current = current.above
                else:
                    current = current.below

            elif current.node_type == 'leaf':
                # Znaleziony trapez
                return current.label

        return None

# Na wykładzie są podane dwa rodzaje tej struktury - musimy wytłumaczyć dlaczego akurat tą wybierami?

In [16]:
# Testy struktury DTree
if __name__ == "__main__":
    # Tworzenie przykładowych danych
    p1 = Point(1, 1)
    p2 = Point(5, 5)
    p3 = Point(3, 2)
    segment = Segment(p1, p2)

    trapez1 = Trapezoid(None, None, p1, p2)
    trapez2 = Trapezoid(None, None, p2, p3)

    leaf1 = DNode('leaf', trapez1)
    leaf2 = DNode('leaf', trapez2)
    x_node = DNode('x', p2, leaf1, leaf2)

    # Tworzenie drzewa
    dtree = DTree()
    dtree.root = x_node

    # Test punktu
    test_point = Point(4, 4)
    result = dtree.query(test_point)
    print("Trapez zawierający punkt:", result.leftp.x, result.rightp.x)

Trapez zawierający punkt: 1 5


## Aplikacja do zadawania odcinków

Aby wprowadzić odcinek należy nacisnąć lewym przyciskiem myszy w punkcie początkowym, przeciągnąć kursor i zwolnić przycisk w punkcie końcowym.
Po wprowadzeniu wszystkich odcinków wystarczy zamknąć okno.

In [17]:
%matplotlib tk

import matplotlib.pyplot as plt

class LineDrawer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.lines = []
        self.current_line = None
        self.start_point = None

        # stałe wymiary okna (można zmienić)
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)

        self.cid_click = self.fig.canvas.mpl_connect('button_press_event', self.on_click)
        self.cid_motion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)
        self.cid_release = self.fig.canvas.mpl_connect('button_release_event', self.on_release)

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        self.start_point = (event.xdata, event.ydata)
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = self.ax.plot([event.xdata, event.xdata], [event.ydata, event.ydata], 'k-')[0]

    def on_motion(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        xdata, ydata = self.current_line.get_data()
        xdata[1], ydata[1] = event.xdata, event.ydata
        self.current_line.set_data(xdata, ydata)
        self.fig.canvas.draw()

    def on_release(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        self.lines.append((self.start_point, (event.xdata, event.ydata)))
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = None
        self.start_point = None
        self.fig.canvas.draw()

    def show(self):
        plt.show()

    def get_lines(self):
        return self.lines

drawer = LineDrawer()
drawer.show()
lines=drawer.get_lines()


In [18]:
print(lines)

[((np.float64(2.4999999999999996), np.float64(7.705627705627706)), (np.float64(6.96774193548387), np.float64(7.034632034632035))), ((np.float64(3.9516129032258065), np.float64(4.329004329004329)), (np.float64(9.080645161290322), np.float64(4.588744588744589)))]


In [19]:
# zamiana reprezentacji odcinków
for i in range(len(lines)):
    p=Point(lines[i][0][0],lines[i][0][1])
    q=Point(lines[i][1][0],lines[i][1][1])
    segment=Segment(p,q)
    lines[i]=segment

## Konstrukcja mapy trapezowej T(S)

wyznaczanie strefy dla $s_i$

In [5]:
def find_area(D,segment):
    area=[]
    p=segment.left
    q=segment.right
    delta=D.query(p)
    area.append(delta)
    while q.x>delta.rightp.x:
        if delta.rightp.y>segment.get_y(delta.rightp.x):
            area.append(delta.lower_right)
            delta=delta.lower_right
        else:
            area.append(delta.upper_right)
            delta=delta.upper_right
    return area
        



In [6]:
def update_left(old_trap, left, top, bottom):
    if left:
        if old_trap.upper_left:
            old_trap.upper_left.upper_right = left
        if old_trap.lower_left:
            old_trap.lower_left.lower_right = left
    else:
        if old_trap.upper_left:
            old_trap.upper_left.upper_right = top
        if old_trap.lower_left:
            old_trap.lower_left.lower_right = bottom


In [7]:
def update_right(old_trap, right, top, bottom):
    if right:
        if old_trap.upper_right:
            old_trap.upper_right.upper_left = right
        if old_trap.lower_right:
            old_trap.lower_right.lower_left = right
    else:
        if old_trap.upper_right:
            old_trap.upper_right.upper_left = top
        if old_trap.lower_right:
            old_trap.lower_right.lower_left = bottom

In [8]:
def insert_into_one_trapezoid(dtree, segment, trapezoid):
    # Funkcja która wstawia odcinek do jednego trapezu

    if segment.left.x > trapezoid.leftp.x:
        left_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, trapezoid.leftp, segment.left)
    else:
        left_trapezoid = None
    
    if segment.right.x < trapezoid.rightp.x:
        right_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, segment.right, trapezoid.rightp)
    else:
        right_trapezoid = None
        
    top_trapezoid = Trapezoid(trapezoid.top, segment, trapezoid.leftp, trapezoid.rightp)
    bottom_trapezoid = Trapezoid(segment, trapezoid.bottom, trapezoid.leftp, trapezoid.rightp)

    
    if left_trapezoid:
        left_trapezoid.upper_right = top_trapezoid
        left_trapezoid.lower_right = bottom_trapezoid
        left_trapezoid.upper_left=trapezoid.upper_left
        left_trapezoid.lower_left=trapezoid.lower_left
        top_trapezoid.upper_left = left_trapezoid
        bottom_trapezoid.lower_left = left_trapezoid
        
    else:
        top_trapezoid.upper_left = trapezoid.upper_left
        bottom_trapezoid.lower_left = trapezoid.lower_left
        
    update_left(trapezoid, left_trapezoid, top_trapezoid, bottom_trapezoid)

    
    if right_trapezoid:
        right_trapezoid.upper_left = top_trapezoid
        right_trapezoid.lower_left = bottom_trapezoid
        right_trapezoid.upper_right=trapezoid.upper_right
        right_trapezoid.lower_right=trapezoid.lower_right
        top_trapezoid.upper_right = right_trapezoid
        bottom_trapezoid.lower_right = right_trapezoid

    else:
        top_trapezoid.upper_right = trapezoid.upper_right
        bottom_trapezoid.lower_right = trapezoid.lower_right

    update_right(trapezoid, right_trapezoid, top_trapezoid, bottom_trapezoid)
            

    # zmiany w dtree
    leftpoint=DNode('x',segment.left)
    if left_trapezoid:
        left=left_trapezoid
    else:
        if trapezoid.upper_left:
            left=trapezoid.upper_left
        else:
            left=trapezoid.lower_left
    leftpoint.left=DNode('leaf',left)
    left.node.parents.append(leftpoint)
    rightpoint=DNode('x',segment.right)
    leftpoint.right=rightpoint
    if right_trapezoid:
        right=right_trapezoid
    else:
        if trapezoid.upper_right:
            right=trapezoid.upper_right
        else:
            right=trapezoid.lower_right
    rightpoint.right=DNode('leaf',right)
    right.node.parents.append(rightpoint)
    segment_node=DNode('y',segment)
    rightpoint.left=segment_node
    segment_node.above=DNode('leaf',top_trapezoid)
    top_trapezoid.node.parents.append(segment_node)
    segment_node.below=DNode('leaf',bottom_trapezoid)
    bottom_trapezoid.node.parents.append(segment_node)
    if dtree.root==trapezoid.node:
        dtree.root=leftpoint
    else:
        trapezoid.node.replace(leftpoint)
    
    


In [9]:
def main():
    outer_trapezoid=Trapezoid(Point(0,0),Point(10,10),Point(0,0),Point(10,0))
    # Tworzenie drzewa
    dtree = DTree()
    # Tworzenie korzenia
    x_node = DNode('leaf', outer_trapezoid)
    dtree.root = x_node
    # zbiór trapezów należących do mapy
    T=set([outer_trapezoid])  # czy to jest potrzebne?
    #lecimy po kolei po odcinkach, po kolei je analizujemy
    for segment in lines:
        
        area=find_area(dtree,segment)
        #patrzymy ile trapezow zajmuje
        n = len( area )
        if n ==1 :
            #tutaj funkcja
            insert_into_one_trapezoid(dtree,segment,area[0],T)
            pass
        elif n > 1 :
            #tutaj funkcja
            pass
    return dtree
        


In [22]:
def randomizer(odcinki):
    # Funkcja która przyjmuje odcinki i je losowo układa
    pass